# 01 - Carga y Limpieza de Datos

**Proyecto:** EDA de Dengue en Colombia  
**Maestría en Inteligencia Artificial** - Desarrollo de Soluciones  

Este notebook carga, inspecciona y limpia los datos de dengue del SIVIGILA y las proyecciones de población del DANE.

**Fuentes de datos:**
- Dengue regular (código 210): años 2010, 2016, 2022, 2024
- Dengue grave (código 220): años 2010, 2016, 2019, 2022, 2024
- Proyecciones de población DANE (2005-2020)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Importar utilidades del proyecto
sys.path.insert(0, os.path.join('..', 'src'))
from utils import (
    cargar_dengue, cargar_dane, configurar_estilo, resumen_dataframe,
    convertir_edad_anos, clasificar_grupo_etario,
    COLS_CLAVE, ANOS_DENGUE_REGULAR, ANOS_DENGUE_GRAVE
)

configurar_estilo()
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)

## 1. Carga de datos de Dengue Regular (código 210)

In [ ]:
print('Cargando datos de Dengue Regular (código 210)...')
print(f'Años disponibles: {ANOS_DENGUE_REGULAR}')
print(f'Nota: No hay datos de dengue regular para 2019\n')

df_dengue = cargar_dengue(tipo='regular')
resumen_dataframe(df_dengue, 'Dengue Regular (210)')

In [ ]:
# Primeras filas
df_dengue.head(3)

In [ ]:
# Tipos de datos
print('Tipos de datos - Dengue Regular:')
print(df_dengue.dtypes.to_string())

## 2. Carga de datos de Dengue Grave (código 220)

In [ ]:
print('Cargando datos de Dengue Grave (código 220)...')
print(f'Años disponibles: {ANOS_DENGUE_GRAVE}\n')

df_grave = cargar_dengue(tipo='grave')
resumen_dataframe(df_grave, 'Dengue Grave (220)')

In [ ]:
df_grave.head(3)

## 3. Carga de datos DANE (Proyecciones de Población)

In [ ]:
print('Cargando proyecciones de población DANE...')
df_dane = cargar_dane()
resumen_dataframe(df_dane, 'Proyecciones DANE')
df_dane.head()

In [ ]:
# Verificar cobertura de departamentos y municipios
print(f'Departamentos DANE: {df_dane["departamento"].nunique()}')
print(f'Municipios DANE: {df_dane["municipio"].nunique()}')
print(f'\nColumnas de población disponibles:')
pob_cols = [c for c in df_dane.columns if c.startswith('pob_')]
print(f'  Años: {[c.replace("pob_", "") for c in pob_cols]}')

## 4. Inspección de estructura y columnas comunes

Verificamos qué columnas son consistentes entre los diferentes años y tipos de dengue.

In [ ]:
# Columnas en dengue regular
print('Columnas en Dengue Regular (210):')
print(f'  Total: {len(df_dengue.columns)}')
print(f'  Lista: {list(df_dengue.columns)}')

print(f'\nColumnas en Dengue Grave (220):')
print(f'  Total: {len(df_grave.columns)}')
print(f'  Lista: {list(df_grave.columns)}')

In [ ]:
# Columnas comunes
cols_comunes = set(df_dengue.columns) & set(df_grave.columns)
cols_solo_regular = set(df_dengue.columns) - set(df_grave.columns)
cols_solo_grave = set(df_grave.columns) - set(df_dengue.columns)

print(f'Columnas comunes: {len(cols_comunes)}')
print(f'Solo en regular: {cols_solo_regular if cols_solo_regular else "Ninguna"}')
print(f'Solo en grave: {cols_solo_grave if cols_solo_grave else "Ninguna"}')

## 5. Limpieza de datos

### 5.1 Verificación de registros por año

In [ ]:
print('Registros por año - Dengue Regular (210):')
print(df_dengue['ANO'].value_counts().sort_index().to_string())

print(f'\nRegistros por año - Dengue Grave (220):')
print(df_grave['ANO'].value_counts().sort_index().to_string())

### 5.2 Conversión de tipos de datos

In [ ]:
# Columnas de fecha
cols_fecha = ['FEC_NOT', 'INI_SIN', 'FEC_HOS', 'FEC_DEF', 'FECHA_NTO', 'FEC_CON', 'FEC_ARC_XL', 'FEC_AJU']

for col in cols_fecha:
    if col in df_dengue.columns:
        df_dengue[col] = pd.to_datetime(df_dengue[col], errors='coerce')
    if col in df_grave.columns:
        df_grave[col] = pd.to_datetime(df_grave[col], errors='coerce')

print('Conversión de fechas completada.')
print(f'\nTipos de columnas de fecha (dengue regular):')
for col in cols_fecha:
    if col in df_dengue.columns:
        print(f'  {col}: {df_dengue[col].dtype}')

In [ ]:
# Convertir columnas numéricas clave
cols_numericas = ['SEMANA', 'ANO', 'EDAD', 'UNI_MED', 'COD_DPTO_O', 'COD_MUN_O',
                  'COD_DPTO_R', 'COD_MUN_R', 'COD_DPTO_N', 'COD_MUN_N']

for col in cols_numericas:
    if col in df_dengue.columns:
        df_dengue[col] = pd.to_numeric(df_dengue[col], errors='coerce')
    if col in df_grave.columns:
        df_grave[col] = pd.to_numeric(df_grave[col], errors='coerce')

print('Conversión de columnas numéricas completada.')

### 5.3 Calcular edad en años

In [ ]:
# Convertir edad a años según unidad de medida SIVIGILA
df_dengue['edad_anos'] = convertir_edad_anos(df_dengue)
df_grave['edad_anos'] = convertir_edad_anos(df_grave)

# Clasificar en grupos etarios
df_dengue['grupo_etario'] = clasificar_grupo_etario(df_dengue['edad_anos'])
df_grave['grupo_etario'] = clasificar_grupo_etario(df_grave['edad_anos'])

print('Edad en años y grupo etario calculados.')
print(f'\nDistribución de edad (años) - Dengue Regular:')
print(df_dengue['edad_anos'].describe())
print(f'\nGrupos etarios - Dengue Regular:')
print(df_dengue['grupo_etario'].value_counts().sort_index())

### 5.4 Análisis de valores nulos

In [ ]:
# Porcentaje de nulos por columna - Dengue Regular
nulos_regular = (df_dengue.isnull().sum() / len(df_dengue) * 100).sort_values(ascending=False)
nulos_regular = nulos_regular[nulos_regular > 0]

print('Columnas con valores nulos - Dengue Regular (210):')
print('(solo columnas con > 0% nulos)\n')
for col, pct in nulos_regular.items():
    print(f'  {col:35s} {pct:6.1f}%  ({df_dengue[col].isnull().sum():,} registros)')

In [ ]:
# Porcentaje de nulos por columna - Dengue Grave
nulos_grave = (df_grave.isnull().sum() / len(df_grave) * 100).sort_values(ascending=False)
nulos_grave = nulos_grave[nulos_grave > 0]

print('Columnas con valores nulos - Dengue Grave (220):')
print('(solo columnas con > 0% nulos)\n')
for col, pct in nulos_grave.items():
    print(f'  {col:35s} {pct:6.1f}%  ({df_grave[col].isnull().sum():,} registros)')

### 5.5 Verificar duplicados

In [ ]:
# Duplicados basados en CONSECUTIVE (identificador único del caso)
dup_regular = df_dengue.duplicated(subset=['CONSECUTIVE'], keep=False).sum()
dup_grave = df_grave.duplicated(subset=['CONSECUTIVE'], keep=False).sum()

print(f'Duplicados por CONSECUTIVE:')
print(f'  Dengue Regular: {dup_regular:,} registros duplicados')
print(f'  Dengue Grave: {dup_grave:,} registros duplicados')

# Si hay duplicados, eliminar manteniendo el primero
if dup_regular > 0:
    antes = len(df_dengue)
    df_dengue = df_dengue.drop_duplicates(subset=['CONSECUTIVE'], keep='first')
    print(f'\n  Dengue Regular: eliminados {antes - len(df_dengue):,} duplicados')

if dup_grave > 0:
    antes = len(df_grave)
    df_grave = df_grave.drop_duplicates(subset=['CONSECUTIVE'], keep='first')
    print(f'  Dengue Grave: eliminados {antes - len(df_grave):,} duplicados')

### 5.6 Verificación de valores de columnas clave

In [ ]:
# Verificar valores de SEXO
print('Valores de SEXO:')
print('  Dengue Regular:', df_dengue['SEXO'].value_counts().to_dict())
print('  Dengue Grave:', df_grave['SEXO'].value_counts().to_dict())

# Verificar TIP_CAS (tipo de caso)
print(f'\nValores de TIP_CAS (tipo de caso):')
print('  Dengue Regular:', df_dengue['TIP_CAS'].value_counts().to_dict())
print('  Dengue Grave:', df_grave['TIP_CAS'].value_counts().to_dict())

# Verificar CON_FIN (condición final)
print(f'\nValores de CON_FIN (condición final):')
print('  Dengue Regular:', df_dengue['CON_FIN'].value_counts().to_dict())
print('  Dengue Grave:', df_grave['CON_FIN'].value_counts().to_dict())

# Verificar PAC_HOS (hospitalización)
print(f'\nValores de PAC_HOS (hospitalización):')
print('  Dengue Regular:', df_dengue['PAC_HOS'].value_counts().to_dict())
print('  Dengue Grave:', df_grave['PAC_HOS'].value_counts().to_dict())

In [ ]:
# Valores de AREA (cabecera/rural)
print('Valores de AREA:')
print('  Dengue Regular:', df_dengue['AREA'].value_counts().to_dict())
print('  Dengue Grave:', df_grave['AREA'].value_counts().to_dict())

# Verificar semana epidemiológica
print(f'\nRango de SEMANA:')
print(f'  Dengue Regular: {df_dengue["SEMANA"].min()} - {df_dengue["SEMANA"].max()}')
print(f'  Dengue Grave: {df_grave["SEMANA"].min()} - {df_grave["SEMANA"].max()}')

## 6. Resumen estadístico (describe)

In [ ]:
print('Resumen estadístico - Dengue Regular (210):')
df_dengue.describe(include='all').T

In [ ]:
print('Resumen estadístico - Dengue Grave (220):')
df_grave.describe(include='all').T

## 7. Visualización rápida de la distribución de datos

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Casos por año - Regular
conteo_reg = df_dengue['ANO'].value_counts().sort_index()
axes[0].bar(conteo_reg.index.astype(str), conteo_reg.values, color='steelblue', edgecolor='white')
axes[0].set_title('Dengue Regular (210) - Casos por Año', fontweight='bold')
axes[0].set_ylabel('Número de casos')
for i, v in enumerate(conteo_reg.values):
    axes[0].text(i, v + v*0.01, f'{v:,}', ha='center', va='bottom', fontsize=9)

# Casos por año - Grave
conteo_grav = df_grave['ANO'].value_counts().sort_index()
axes[1].bar(conteo_grav.index.astype(str), conteo_grav.values, color='firebrick', edgecolor='white')
axes[1].set_title('Dengue Grave (220) - Casos por Año', fontweight='bold')
axes[1].set_ylabel('Número de casos')
for i, v in enumerate(conteo_grav.values):
    axes[1].text(i, v + v*0.01, f'{v:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Visualización de nulos como heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Solo columnas con al menos 1% de nulos
cols_nulos_reg = df_dengue.columns[df_dengue.isnull().mean() > 0.01]
if len(cols_nulos_reg) > 0:
    nulos_por_ano_reg = df_dengue.groupby('ANO')[cols_nulos_reg].apply(lambda x: x.isnull().mean() * 100)
    sns.heatmap(nulos_por_ano_reg.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[0])
    axes[0].set_title('% Nulos por Año - Dengue Regular', fontweight='bold')
else:
    axes[0].text(0.5, 0.5, 'Sin columnas con >1% nulos', ha='center', va='center')
    axes[0].set_title('Dengue Regular - Sin nulos significativos')

cols_nulos_grav = df_grave.columns[df_grave.isnull().mean() > 0.01]
if len(cols_nulos_grav) > 0:
    nulos_por_ano_grav = df_grave.groupby('ANO')[cols_nulos_grav].apply(lambda x: x.isnull().mean() * 100)
    sns.heatmap(nulos_por_ano_grav.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1])
    axes[1].set_title('% Nulos por Año - Dengue Grave', fontweight='bold')
else:
    axes[1].text(0.5, 0.5, 'Sin columnas con >1% nulos', ha='center', va='center')
    axes[1].set_title('Dengue Grave - Sin nulos significativos')

plt.tight_layout()
plt.show()

## 8. Resumen de la carga y limpieza

### Hallazgos principales

In [ ]:
print('='*60)
print('  RESUMEN DE CARGA Y LIMPIEZA')
print('='*60)

print(f'\nDengue Regular (código 210):')
print(f'  Registros totales: {len(df_dengue):,}')
print(f'  Años: {sorted(df_dengue["ANO"].unique())}')
print(f'  Columnas: {len(df_dengue.columns)}')
print(f'  Nota: No hay datos para 2019')

print(f'\nDengue Grave (código 220):')
print(f'  Registros totales: {len(df_grave):,}')
print(f'  Años: {sorted(df_grave["ANO"].unique())}')
print(f'  Columnas: {len(df_grave.columns)}')

print(f'\nDANE - Proyecciones de Población:')
print(f'  Municipios: {len(df_dane):,}')
print(f'  Departamentos: {df_dane["departamento"].nunique()}')
print(f'  Años de población: 2005-2020')
print(f'  Nota: Para 2022 y 2024 se usará la proyección de 2020 como aproximación')

print(f'\nDatos climáticos:')
print(f'  Pendiente de ejecución del notebook 00_descarga_clima_gee.ipynb')